# Let's take a look at the General Service List (list of common english words) ... 2000 words or so

https://www.eapfoundation.com/vocab/general/gsl/

# Corpus of Contemporary American English (COCA) <-- sorted by frequency 
https://www.wordfrequency.info/ 
UGH i have to purchase this? booo

# Game plan...
* I want certain reactive groups to correlate with whether things are nouns or verbs.
    * I think that means I have to somehow separate out reactive vs. unreactive chemicals first.
    * Similarly separate out verbs and nouns in english (that should have a function already)
* Then I want to maybe match the most common words in each category using COCA for english and some other sort of frequency of most common chemicals...
* After frequency, let's rely on embeddings to get everything else closest to the nearest neighbor anchor point. This way we can be sure things are at least somewhat aligned.
    * alternatively we don't even start with the anchors but jump straight into the embeddings? Need to talk to David to get his help here with alignment.


* Subjunctive / conditional / other fun tenses could be inert gases that I add to the reaction like Helium could be a good example of conditional.
* Electro chemistry would be crazy... maybe that can be for a tense that is crazy... again I want to say conditional.

* Past, present, future will be decided by which side of a reaction is given. if it's in equilibrium then it's present.

I guess stoichometry is important here :^P

* One thing I wish would be that we have 

In [1]:
import pandas as pd
import torch
from transformers import BertModel, BertTokenizer


# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


def get_word_embeddings(words):
    inputs = tokenizer(words, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

: 

In [ ]:
words_df[words_df['POS'] == 'Noun']


# GETTING A LIST OF WORDS (Dont need to rerun)

In [1]:
import nltk
# nltk.download('bnc')
nltk.download('brown')
nltk.download('gutenberg')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# POS tag descriptions based on the Penn Treebank tag set
pos_tag_descriptions = {
    "CC": "Coordinating conjunction",
    "CD": "Cardinal number",
    "DT": "Determiner",
    "EX": "Existential there",
    "FW": "Foreign word",
    "IN": "Preposition or subordinating conjunction",
    "JJ": "Adjective",
    "JJR": "WEIRD Adjective",
    "JJS": "WEIRD Adjective",
    "LS": "List item marker",
    "MD": "Modal",
    "NN": "Noun",
    "NNS": "Noun",
    "NNP": "Noun", # Proper Noun
    "NNPS": "Noun",
    "PDT": "Predeterminer",
    "POS": "Possessive ending",
    "PRP": "Noun", # Personal pronoun
    "PRP$": "Noun", # Possessive pronoun
    "RB": "Adverb",
    "RBR": "Adverb",
    "RBS": "Adverb",
    "RP": "Particle",
    "SYM": "Symbol",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb",
    "VBD": "Verb",
    "VBG": "Verb",
    "VBN": "Verb",
    "VBP": "Verb",
    "VBZ": "Verb",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive wh-pronoun",
    "WRB": "Wh-adverb",
    "``": "Punctuation",
    "''": "Punctuation",
    ",": "Punctuation",
    ".": "Punctuation",
    ":": "Punctuation",
    "(": "Punctuation",
    ")": "Punctuation",
    "$": "Punctuation",
    "#": "Punctuation",
}

In [3]:
from nltk.corpus import brown, gutenberg
from nltk import FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import re

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to determine the lemma based on POS tag
def get_lemma(word, tag):
    # Exclude words that are entirely numeric or symbols
    if re.fullmatch(r'[\d$.,]+', word):
        return None  # Return None to indicate exclusion
    
    # Handle possessive cases
    if word.endswith("'s") or word.endswith("’s"):
        word = word[:-2]  # Remove 's
    elif word.endswith("'") or word.endswith("’"):
        word = word[:-1]  # Remove trailing apostrophe

    if tag.startswith('NN'):
        return lemmatizer.lemmatize(word, pos='n')  # for nouns
    elif tag.startswith('VB'):
        return lemmatizer.lemmatize(word, pos='v')  # for verbs
    elif tag.startswith('JJ'):
        return lemmatizer.lemmatize(word, pos='a')  # for adjectives
    elif tag.startswith('RB'):
        return lemmatizer.lemmatize(word, pos='r')  # for adverbs
    else:
        return word
    

words = [word.lower() for word in brown.words()] + [word.lower() for word in gutenberg.words()]
tagged_words = pos_tag(words)

In [4]:
tagged_lemmas = [
    (get_lemma(word, pos), pos_tag_descriptions[pos]) for (word, pos) in tagged_words
]
total_freq_dist = FreqDist(tagged_lemmas)
sorted_words_by_POS = {}
for (lemma, pos), count in total_freq_dist.items():
    sorted_words_by_POS[pos] = sorted_words_by_POS.get(pos, []) + [(lemma, count)]

for pos, word_list in sorted_words_by_POS.items():
    sorted_words_by_POS[pos] = sorted(word_list, key=lambda item: item[1], reverse=True)


In [5]:
len(sorted_words_by_POS['Noun'])

38590

In [6]:
sorted_words_by_POS['Noun'][3000:3100]

[('deficiency', 39),
 ('trustee', 39),
 ('output', 39),
 ('lecture', 39),
 ('generosity', 39),
 ('capability', 39),
 ('earnest', 39),
 ('stride', 39),
 ('contradiction', 39),
 ('straw', 39),
 ('gap', 39),
 ('spoken', 39),
 ('melody', 39),
 ('understood', 39),
 ('shakespeare', 39),
 ('sarah', 39),
 ('deceit', 39),
 ('worthy', 39),
 ('usage', 39),
 ('anyhow', 39),
 ('sinai', 39),
 ('spray', 39),
 ('pig', 39),
 ('spoon', 39),
 ('hide', 39),
 ('intercourse', 39),
 ('hang', 39),
 ('cavalry', 39),
 ('knot', 39),
 ('orphan', 39),
 ('pilate', 39),
 ('cheerfulness', 39),
 ('sprang', 39),
 ('watchman', 39),
 ('shipmate', 39),
 ('perfect', 39),
 ('musgroves', 39),
 ('ishmael', 39),
 ('merari', 39),
 ('jehoshaphat', 39),
 ('mordecai', 39),
 ('whaler', 39),
 ('downe', 39),
 ('rosin', 39),
 ('r.', 38),
 ('associate', 38),
 ('outcome', 38),
 ('latin', 38),
 ('admission', 38),
 ('graduate', 38),
 ('steward', 38),
 ('knight', 38),
 ('holder', 38),
 ('jump', 38),
 ('proprietor', 38),
 ('lace', 38),
 ('e

In [10]:
import pandas as pd 

data = []
for pos, word_list in sorted_words_by_POS.items():
    for word, frequency in word_list:
        data.append({
            'Word': word,
            'Frequency': frequency,
            'POS': pos
        })

words_df = pd.DataFrame(data)
words_df

,Word,Frequency,POS
0,the,203554,Determiner
1,a,57155,Determiner
2,this,14060,Determiner
3,all,11048,Determiner
4,an,8638,Determiner
...,...,...,...
81705,banner,1,Symbol
81706,non,1,Symbol
81707,shakspere,1,Symbol
81708,oak,1,Symbol


In [11]:
words_df.to_csv('words.csv')

In [13]:
words_df[words_df['POS'] == 'Noun']


,Word,Frequency,POS
102,he,35466,Noun
103,it,31128,Noun
104,his,28398,Noun
105,you,19684,Noun
106,i,18946,Noun
...,...,...,...
38687,estray,1,Noun
38688,gluttony,1,Noun
38689,dissolute,1,Noun
38690,shopfront,1,Noun
